In [0]:
# このファイルをAzure Databricksにインポートする

In [0]:
# プログラムの実行

In [0]:
from azureml.core import Workspace

ws = Workspace(workspace_name='Azureml-SDK-WS01',
               subscription_id='3467f739-a57b-4612-9de8-72a6616c01b3',
               resource_group='AzuremlSDKRG00')

input_ds = ws.datasets.get('AdultIncome')

from azureml.core import Experiment
experiment = Experiment(workspace=ws,
                        name="demoexp001")

new_run = experiment.start_logging()

df = input_ds.to_pandas_dataframe()
total_observations = len(df)
nulldf = df.isnull().sum()

new_run.log("Total Observations", total_observations)
for columns in df.columns:
    new_run.log(columns, nulldf[columns])
    
new_run.complete()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HN8BFZDLX to authenticate.
Interactive authentication successfully completed.

In [0]:
# Storageアカウントのマウント
# Storageアカウントを作成しておく

In [0]:
source = "wasbs://mount@azuremlsstorage74437b7d7.blob.core.windows.net/"
mount_point = "/mnt/aidata"
# azuremlsstorage の部分はStorageのアカウント名
conf_key = "fs.azure.account.key.azuremlsstorage74437b7d7.blob.core.windows.net"
# ストレージアカウントのキー
key_value = "Ao2xKqEIDIPGbybEGIl1s+4rWuagEVEqV+2TaGKAxa7sJtrN8ssq/iBB3hCltPRUTsIRffIf+rTd+AStoeKjwQ=="

In [0]:
# 既にマウントしていれば一度解除する
try:
    dbutils.fs.unmount(mount_point)
except:
    print("The mount did not exist.")

The mount did not exist.

In [0]:
# マウントを実行
dbutils.fs.mount(
  source = source,
  mount_point = mount_point,
  extra_configs = {conf_key:key_value})

Out[14]: True

In [0]:
import pandas as pd
# dbfsはファイルシステム
df = pd.read_csv("/dbfs/mnt/aidata/adultincome trunc.csv")
df.head()

Out[19]:

,age,wc,education,marital status,race,gender,hours per week,IncomeClass
0,38,Private,HS-grad,Divorced,White,Male,40,<=50K
1,28,Private,Bachelors,Married,Black,Female,40,<=50K
2,37,Private,Masters,Married,White,Female,40,<=50K
3,31,Private,Masters,Never-married,White,Female,50,>50K
4,42,Private,Bachelors,Married,White,Male,40,>50K


In [0]:
# ここからはDatabricksの効率的な使い方を見ていく
# Cluster ⇒ Libraries ⇒ PYPI
# scikit-learn を入力する

In [0]:
# これはstorageアカウントにdatafolderフォルダが無いのでエラーになる
df.to_csv("/dbfs/mnt/aidata/datafolder/adulttest.csv")

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-1108921059205738> in <module> 
 ----> 1 df . to_csv ( "/dbfs/mnt/aidata/datafolder/adulttest.csv" ) 

 /databricks/python/lib/python3.8/site-packages/pandas/core/generic.py in to_csv (self, path_or_buf, sep, na_rep, float_format, columns, header, index, index_label, mode, encoding, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, decimal, errors, storage_options) 
 3385 )
 3386 
 -> 3387 return DataFrameRenderer(formatter).to_csv(
 3388 path_or_buf , 
 3389 line_terminator = line_terminator , 

 /databricks/python/lib/python3.8/site-packages/pandas/io/formats/format.py in to_csv (self, path_or_buf, encoding, sep, columns, index_label, mode, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, errors, storage_options) 
 1081 formatter = self . fmt , 
 1082 )
 -> 1083 csv_formatter . save ( ) 
 1084 
 1085 if created_buffer : 

 /databricks/python/lib/python3.8/site-packages/pandas/io/formats/csvs.py in save (self) 
 226 """
 227 # apply compression and byte/text conversion 
 --> 228 with get_handle(
 229 self . filepath_or_buffer , 
 230 self . mode , 

 /databricks/python/lib/python3.8/site-packages/pandas/io/common.py in get_handle (path_or_buf, mode, encoding, compression, memory_map, is_text, errors, storage_options) 
 640 errors = "replace" 
 641 # Encoding 
 --> 642 handle = open(
 643 handle , 
 644 ioargs . mode , 

 FileNotFoundError : [Errno 2] No such file or directory: '/dbfs/mnt/aidata/datafolder/adulttest.csv'

In [0]:
# sparkdfを作成すると保存できる
# storageアカウントに見に行くとフォルダの中にadultspark.csvの中にいろんなファイルがある
# 複数ファイルに分割されている
spark_df = spark.createDataFrame(df)
spark_df.write.csv(path="/mnt/aidata/sparkfolder/adultspark.csv", mode="overwrite")

In [0]:
# 一つのファイルにしたいときは以下
dbutils.fs.mkdirs("/mnt/aidata/datafolder/000") # blobには何も入っていないフォルダは作成できないので000をつける
df.to_csv("/dbfs/mnt/aidata/datafolder/adulttest.csv")

In [0]:
# パラメータをNotebook間でやり取りする

In [0]:
# パラメータを他のNotebookとやり取りする
# %runコマンドのセルにはコメントがあるとだめ

# 以下のコードをnb002として作っておく
# p1 = dbutils.widgets.get("p1")
# print("Inside nb002 ", p1)

In [0]:
%run ./nb002 $p1="Hello"

Inside nb002 Hello